# 📘 FILE 1-C – tf.keras & First Neural Network

## 🎯 Mục tiêu

Sau bài này bạn sẽ hiểu:
- **tf.keras** là gì và vì sao nó quan trọng
- **Sequential API** - cách build model đơn giản nhất
- **Dense layer** - layer cơ bản nhất
- **Activation functions** (ReLU, Softmax, Sigmoid)
- Build và train Neural Network đầu tiên

---

## 📌 Tại sao học tf.keras?

- **High-level API** - dễ dùng, code ngắn gọn
- **Industry standard** - 90% project dùng Keras
- **Tích hợp sẵn** trong TensorFlow 2.x
- **Production-ready** - từ research → deploy

---

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__}")

---

## 1️⃣ tf.keras là gì?

### 🔹 Lịch sử ngắn gọn

- **Keras** (ban đầu): Thư viện độc lập, chạy trên nhiều backend (TensorFlow, Theano, CNTK)
- **tf.keras** (hiện tại): Keras được tích hợp chính thức vào TensorFlow 2.x

### 🔹 tf.keras vs Keras

```python
# Keras cũ (deprecated)
from keras.models import Sequential

# tf.keras (chuẩn hiện tại)
from tensorflow.keras.models import Sequential
# hoặc
tf.keras.models.Sequential
```

**👉 Luôn dùng `tf.keras`!**

### 🔹 3 level API của TensorFlow

```
High-level (dễ):   tf.keras (⭐ recommend)
Mid-level:         tf.nn, tf.layers
Low-level (khó):   tf.GradientTape + manual
```

---

---

## 2️⃣ Sequential API - Build model đơn giản

### 🔹 Sequential là gì?

**Sequential** = model có các layer xếp chồng lên nhau theo thứ tự

```
Input → Layer 1 → Layer 2 → ... → Output
```

### 🔹 Cú pháp

**Cách 1: List các layer**

```python
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])
```

**Cách 2: Add từng layer**

```python
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1))
```

---

In [ ]:
# Ví dụ 1: Model đơn giản nhất
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1)  # 1 neuron, no activation
])

print(model.summary())

In [ ]:
# Ví dụ 2: Multi-layer model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', name='hidden1'),
    tf.keras.layers.Dense(32, activation='relu', name='hidden2'),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
])

print(model.summary())

### 🔹 Hiểu model.summary()

```
Layer (type)                Output Shape              Param #
================================================================
hidden1 (Dense)            (None, 64)                ???
```

**Output Shape:**
- `(None, 64)` → batch_size không cố định, 64 neurons

**Param #:**
- Số parameters (weights + biases)
- Công thức: `(input_dim + 1) × output_dim`

---

In [ ]:
# Tính số parameters
# Giả sử input_dim = 10
input_dim = 10
output_dim = 64

# Dense layer: y = Wx + b
# W shape: (input_dim, output_dim)
# b shape: (output_dim,)
num_weights = input_dim * output_dim
num_biases = output_dim
total_params = num_weights + num_biases

print(f"Number of weights: {num_weights}")
print(f"Number of biases: {num_biases}")
print(f"Total parameters: {total_params}")
print(f"Formula: ({input_dim} + 1) × {output_dim} = {(input_dim + 1) * output_dim}")

---

## 3️⃣ Dense Layer - Layer cơ bản nhất

### 🔹 Dense layer là gì?

**Dense** = Fully Connected Layer = mọi neuron đều kết nối với mọi neuron ở layer trước

### 🔹 Công thức

```
output = activation(dot(input, W) + b)
```

Trong đó:
- `input`: shape `(batch_size, input_dim)`
- `W`: weights, shape `(input_dim, units)`
- `b`: biases, shape `(units,)`
- `activation`: hàm kích hoạt (ReLU, Sigmoid...)

### 🔹 Parameters

```python
tf.keras.layers.Dense(
    units,                    # Số neurons (output dimension)
    activation=None,          # Hàm kích hoạt
    use_bias=True,           # Có dùng bias không
    kernel_initializer='glorot_uniform',  # Cách khởi tạo weights
    bias_initializer='zeros',             # Cách khởi tạo biases
    name=None                # Tên layer (optional)
)
```

---

In [ ]:
# Ví dụ: Dense layer chi tiết
layer = tf.keras.layers.Dense(
    units=5,
    activation='relu',
    name='my_dense_layer'
)

# Tạo input giả
input_data = tf.random.normal([3, 10])  # batch_size=3, input_dim=10

# Forward pass
output = layer(input_data)

print(f"Input shape: {input_data.shape}")
print(f"Output shape: {output.shape}")
print(f"\nWeights shape: {layer.weights[0].shape}")
print(f"Biases shape: {layer.weights[1].shape}")
print(f"\nTotal parameters: {layer.count_params()}")

### 🔹 Visualization: Dense Layer

In [ ]:
# Visualize weights
weights = layer.weights[0].numpy()  # Shape: (10, 5)

plt.figure(figsize=(10, 6))
plt.imshow(weights, cmap='coolwarm', aspect='auto')
plt.colorbar(label='Weight value')
plt.xlabel('Output neurons (5)')
plt.ylabel('Input features (10)')
plt.title('Dense Layer Weights Visualization')
plt.show()

print("Mỗi cột = weights của 1 neuron")
print("Mỗi hàng = weights cho 1 input feature")

---

## 4️⃣ Activation Functions

### 🔹 Tại sao cần activation?

**Không có activation:**
```
Layer1: y1 = W1*x + b1
Layer2: y2 = W2*y1 + b2 = W2*(W1*x + b1) + b2
       = (W2*W1)*x + (W2*b1 + b2)
```

→ Nhiều layer vẫn chỉ là **linear transformation**!

**Có activation:**
```
Layer1: y1 = activation(W1*x + b1)
Layer2: y2 = activation(W2*y1 + b2)
```

→ Model có khả năng học **non-linear patterns**!

### 🔹 Các activation phổ biến

---

In [ ]:
# Generate x for plotting
x = np.linspace(-5, 5, 200)

# Compute activations
relu = np.maximum(0, x)
sigmoid = 1 / (1 + np.exp(-x))
tanh = np.tanh(x)

# Softmax (special case for visualization)
# We'll show it separately

plt.figure(figsize=(15, 4))

# ReLU
plt.subplot(1, 3, 1)
plt.plot(x, relu, 'b-', linewidth=2)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.title('ReLU: max(0, x)')
plt.xlabel('x')
plt.ylabel('ReLU(x)')

# Sigmoid
plt.subplot(1, 3, 2)
plt.plot(x, sigmoid, 'g-', linewidth=2)
plt.grid(True, alpha=0.3)
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.3, label='0.5')
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.title('Sigmoid: 1/(1+e^-x)')
plt.xlabel('x')
plt.ylabel('Sigmoid(x)')
plt.legend()

# Tanh
plt.subplot(1, 3, 3)
plt.plot(x, tanh, 'r-', linewidth=2)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.title('Tanh')
plt.xlabel('x')
plt.ylabel('Tanh(x)')

plt.tight_layout()
plt.show()

### 🔹 1. ReLU (Rectified Linear Unit)

```python
ReLU(x) = max(0, x)
```

**Đặc điểm:**
- `x > 0` → giữ nguyên
- `x ≤ 0` → 0

**Khi nào dùng:**
- **Hidden layers** (default choice)
- Regression problems

**Ưu điểm:**
- Đơn giản, nhanh
- Tránh vanishing gradient

**Nhược điểm:**
- Dying ReLU (neurons = 0 mãi mãi)

---

In [ ]:
# ReLU trong TensorFlow
x = tf.constant([-2.0, -1.0, 0.0, 1.0, 2.0])
relu_output = tf.nn.relu(x)

print(f"Input:  {x.numpy()}")
print(f"Output: {relu_output.numpy()}")

### 🔹 2. Sigmoid

```python
Sigmoid(x) = 1 / (1 + e^(-x))
```

**Đặc điểm:**
- Output: `(0, 1)`
- Smooth curve

**Khi nào dùng:**
- **Binary classification** (output layer)
- Xác suất (0-1)

**Ưu điểm:**
- Output có ý nghĩa (probability)

**Nhược điểm:**
- Vanishing gradient
- Chậm hơn ReLU

---

In [ ]:
# Sigmoid trong TensorFlow
x = tf.constant([-2.0, -1.0, 0.0, 1.0, 2.0])
sigmoid_output = tf.nn.sigmoid(x)

print(f"Input:  {x.numpy()}")
print(f"Output: {sigmoid_output.numpy()}")
print("\nNhận xét:")
print("  x = 0  → sigmoid = 0.5")
print("  x > 0  → sigmoid > 0.5")
print("  x < 0  → sigmoid < 0.5")

### 🔹 3. Softmax

```python
Softmax(x_i) = e^(x_i) / sum(e^(x_j))
```

**Đặc điểm:**
- Output: `(0, 1)` cho mỗi class
- **Sum of all outputs = 1** (probability distribution)

**Khi nào dùng:**
- **Multi-class classification** (output layer)
- Cần probability cho nhiều class

**Ví dụ:**
```
Input:  [2.0, 1.0, 0.1]
Output: [0.66, 0.24, 0.10]  # Sum = 1.0
```

---

In [ ]:
# Softmax trong TensorFlow
logits = tf.constant([[2.0, 1.0, 0.1]])
softmax_output = tf.nn.softmax(logits)

print(f"Input (logits): {logits.numpy()}")
print(f"Output (probs): {softmax_output.numpy()}")
print(f"Sum: {tf.reduce_sum(softmax_output).numpy():.6f}")

# Visualization
classes = ['Cat', 'Dog', 'Bird']
probs = softmax_output.numpy()[0]

plt.figure(figsize=(8, 5))
plt.bar(classes, probs, color=['blue', 'green', 'red'], alpha=0.7)
plt.ylabel('Probability')
plt.title('Softmax Output Example')
plt.ylim([0, 1])
for i, (cls, prob) in enumerate(zip(classes, probs)):
    plt.text(i, prob + 0.02, f'{prob:.2f}', ha='center')
plt.show()

### 🔹 Cheat Sheet: Khi nào dùng activation nào?

| Task | Hidden Layers | Output Layer |
|------|---------------|-------------|
| Regression | ReLU | None (linear) |
| Binary Classification | ReLU | Sigmoid |
| Multi-class Classification | ReLU | Softmax |
| Image Generation | ReLU | Tanh / Sigmoid |

---

---

## 5️⃣ First Neural Network - Regression

### 🔹 Bài toán: Predict House Price

```
Input:  [area, bedrooms, age]
Output: price
```

---

In [ ]:
# Generate fake data
np.random.seed(42)

num_samples = 1000

# Features: [area (m²), bedrooms, age (years)]
area = np.random.uniform(50, 200, num_samples)
bedrooms = np.random.randint(1, 6, num_samples)
age = np.random.uniform(0, 50, num_samples)

# True relationship: price = 1000*area + 50000*bedrooms - 500*age + noise
price = 1000 * area + 50000 * bedrooms - 500 * age + np.random.randn(num_samples) * 10000

# Combine features
X = np.column_stack([area, bedrooms, age]).astype(np.float32)
y = price.reshape(-1, 1).astype(np.float32)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nFirst 5 samples:")
print("Area\tBeds\tAge\tPrice")
for i in range(5):
    print(f"{X[i, 0]:.1f}\t{X[i, 1]:.0f}\t{X[i, 2]:.1f}\t${y[i, 0]:,.0f}")

In [ ]:
# Split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train size: {X_train.shape[0]}")
print(f"Test size: {X_test.shape[0]}")

In [ ]:
# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(3,), name='hidden1'),
    tf.keras.layers.Dense(32, activation='relu', name='hidden2'),
    tf.keras.layers.Dense(1, name='output')  # No activation for regression
], name='house_price_model')

print(model.summary())

In [ ]:
# Compile model
model.compile(
    optimizer='adam',           # Optimizer (sẽ học kỹ sau)
    loss='mse',                 # Mean Squared Error
    metrics=['mae']             # Mean Absolute Error
)

print("Model compiled!")

In [ ]:
# Train model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=0  # Tắt log chi tiết
)

print("Training completed!")

In [ ]:
# Plot training history
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on test set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss (MSE): {test_loss:,.0f}")
print(f"Test MAE: ${test_mae:,.0f}")

# Make predictions
y_pred = model.predict(X_test, verbose=0)

# Plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
plt.xlabel('Actual Price ($)')
plt.ylabel('Predicted Price ($)')
plt.title('Predictions vs Actual')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Test on new data
new_house = np.array([[100, 3, 10]])  # 100m², 3 bedrooms, 10 years old

predicted_price = model.predict(new_house, verbose=0)[0, 0]

print(f"House: {new_house[0, 0]:.0f}m², {new_house[0, 1]:.0f} bedrooms, {new_house[0, 2]:.0f} years old")
print(f"Predicted price: ${predicted_price:,.0f}")

---

## 6️⃣ First Neural Network - Classification

### 🔹 Bài toán: Binary Classification (Cancer Detection)

```
Input:  30 medical features
Output: 0 (benign) or 1 (malignant)
```

---

In [ ]:
# Load dataset
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

data = load_breast_cancer()
X = data.data.astype(np.float32)
y = data.target.astype(np.float32).reshape(-1, 1)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nClasses: {np.unique(y)}")
print(f"Class distribution:")
print(f"  Benign (0): {(y == 0).sum()}")
print(f"  Malignant (1): {(y == 1).sum()}")

In [ ]:
# Normalize features (QUAN TRỌNG cho Neural Network!)
scaler = StandardScaler()
X = scaler.fit_transform(X).astype(np.float32)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train size: {X_train.shape[0]}")
print(f"Test size: {X_test.shape[0]}")

In [ ]:
# Build model
model_clf = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(30,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification!
], name='cancer_classifier')

print(model_clf.summary())

In [ ]:
# Compile
model_clf.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary classification loss
    metrics=['accuracy']
)

# Train
history_clf = model_clf.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

print("Training completed!")

In [ ]:
# Plot history
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history_clf.history['loss'], label='Train Loss')
plt.plot(history_clf.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Binary Crossentropy Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_clf.history['accuracy'], label='Train Acc')
plt.plot(history_clf.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate
test_loss, test_acc = model_clf.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Predictions
y_pred_prob = model_clf.predict(X_test, verbose=0)
y_pred = (y_pred_prob > 0.5).astype(int)

# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 5))
plt.imshow(cm, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.xticks([0, 1], ['Benign', 'Malignant'])
plt.yticks([0, 1], ['Benign', 'Malignant'])

# Add text
for i in range(2):
    for j in range(2):
        plt.text(j, i, str(cm[i, j]), ha='center', va='center', fontsize=20)

plt.show()

print(f"\nTrue Negatives: {cm[0, 0]}")
print(f"False Positives: {cm[0, 1]}")
print(f"False Negatives: {cm[1, 0]}")
print(f"True Positives: {cm[1, 1]}")

---

## 7️⃣ Model Methods - Tổng hợp

### 🔹 Các method quan trọng

```python
# 1. Build model
model = tf.keras.Sequential([...])

# 2. Compile (cấu hình training)
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

# 3. Train
history = model.fit(X_train, y_train, epochs=10)

# 4. Evaluate
loss, metric = model.evaluate(X_test, y_test)

# 5. Predict
predictions = model.predict(X_new)

# 6. Save/Load
model.save('my_model.keras')
model = tf.keras.models.load_model('my_model.keras')
```

---

In [ ]:
# Demo save/load
model_clf.save('/tmp/cancer_model.keras')
print("Model saved!")

# Load
loaded_model = tf.keras.models.load_model('/tmp/cancer_model.keras')
print("Model loaded!")

# Test loaded model
test_acc_loaded = loaded_model.evaluate(X_test, y_test, verbose=0)[1]
print(f"Loaded model accuracy: {test_acc_loaded:.4f}")

---

## 8️⃣ Common Mistakes

### ❌ Mistake 1: Quên activation ở hidden layer

In [ ]:
# SAI - Không có activation
bad_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64),  # THIẾU activation!
    tf.keras.layers.Dense(32),  # THIẾU activation!
    tf.keras.layers.Dense(1)
])

print("Model này chỉ là linear transformation!")
print("Không thể học non-linear patterns!")

In [ ]:
# ĐÚNG - Có activation
good_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

print("Model này có thể học non-linear patterns!")

### ❌ Mistake 2: Sai activation ở output layer

In [ ]:
# SAI - Dùng ReLU cho binary classification
bad_clf = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')  # SAI!
])

print("Output không phải probability (0-1)!")
print("Phải dùng sigmoid cho binary classification!")

In [ ]:
# ĐÚNG
good_clf = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # ĐÚNG!
])

print("Output là probability (0-1)!")

### ❌ Mistake 3: Quên normalize data

In [ ]:
# Ví dụ: Features có scale khác nhau
X_bad = np.array([
    [100000, 3, 10],  # [price, bedrooms, age]
    [200000, 4, 5],
    [150000, 2, 20]
])

print("Feature 1 (price): 100,000 - 200,000")
print("Feature 2 (beds): 2 - 4")
print("Feature 3 (age): 5 - 20")
print("\nScale khác nhau → training khó khăn!")

# ĐÚNG: Normalize
scaler = StandardScaler()
X_good = scaler.fit_transform(X_bad)

print("\nAfter normalization:")
print(X_good)
print("\nMean ≈ 0, Std ≈ 1 → training dễ hơn!")

---

## 9️⃣ Exercises

### 📝 Exercise 1: Multi-class Classification

Build model để phân loại iris dataset (3 classes)

**Hints:**
- Output layer: `Dense(3, activation='softmax')`
- Loss: `'sparse_categorical_crossentropy'`

In [ ]:
# YOUR CODE HERE
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target.astype(np.int32)

# TODO: Normalize X
# TODO: Split data
# TODO: Build model (3 classes!)
# TODO: Compile
# TODO: Train
# TODO: Evaluate

### 📝 Exercise 2: Deeper Network

Thử nghiệm với model có nhiều layer hơn

**Hints:**
- Thử 5-6 hidden layers
- Giảm dần số neurons: 128 → 64 → 32 → 16

In [ ]:
# YOUR CODE HERE
# TODO: Build deeper model
# TODO: Compare với model nông (2 layers)
# TODO: Vẽ learning curves

### 📝 Exercise 3: Custom Activation

Implement Swish activation: `swish(x) = x * sigmoid(x)`

**Hints:**
```python
def swish(x):
    return x * tf.nn.sigmoid(x)

tf.keras.layers.Dense(64, activation=swish)
```

In [ ]:
# YOUR CODE HERE
# TODO: Implement swish
# TODO: Build model với swish
# TODO: So sánh với ReLU

---

## 🎯 Tóm tắt

### ✅ Những gì đã học

1. **tf.keras**: High-level API cho TensorFlow
2. **Sequential**: Build model đơn giản (layer stack)
3. **Dense**: Fully connected layer
4. **Activations**: ReLU (hidden), Sigmoid (binary), Softmax (multi-class)
5. **Training flow**: Compile → Fit → Evaluate → Predict

### 🎓 Key Takeaways

**Pattern chuẩn:**

```python
# 1. Build
model = tf.keras.Sequential([
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(output_dim, activation=output_activation)
])

# 2. Compile
model.compile(optimizer='adam', loss=..., metrics=[...])

# 3. Train
model.fit(X_train, y_train, epochs=50, validation_split=0.2)

# 4. Evaluate
model.evaluate(X_test, y_test)
```

### 📚 Next Steps

- **File 1-D**: GPU, Debugging & Final Exercises

---

## 📖 References

- [Keras Sequential Guide](https://www.tensorflow.org/guide/keras/sequential_model)
- [Keras Layers Reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers)
- [Activation Functions](https://www.tensorflow.org/api_docs/python/tf/keras/activations)

---

**Chúc bạn học tốt! 🚀**